In [1]:
import logging
import time
import redis
import requests
import re,json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

In [2]:
##   var domain_active 定义在页面首页
domain_def_list =[{"id":"2a","name":".com 英文域名","url":"http://wanwang.aliyun.com/domain/com/"},{"id":"15201","name":".net 英文域名","url":"http://wanwang.aliyun.com/domain/net/"},{"id":"19922","name":".cn 英文域名","url":"http://wanwang.aliyun.com/domain/cn/"},{"id":"2a1","name":".info 信息域名","url":"http://wanwang.aliyun.com/domain/info/"},{"id":"15527","name":".xin 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.xin"},{"id":"12259","name":".club 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.club"},{"id":"12257","name":".xyz 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.xyz"},{"id":"16064","name":".ltd 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.ltd"},{"id":"12254","name":".wang 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.wang"},{"id":"12255","name":".top 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.top"},{"id":"16054","name":".vip 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.vip"},{"id":"180824","name":".beer 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#beer"},{"id":"17081","name":".ink 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.ink"},{"id":"15092","name":".site 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/#.site"},{"id":"16065","name":".shop 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/#.shop"},{"id":"171001","name":".fun 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.fun"},{"id":"15072","name":".link 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.link"},{"id":"15088","name":".online 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.online"},{"id":"15091","name":".tech 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.tech"},{"id":"12258","name":".ren 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.ren"},{"id":"16091","name":".pro 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.pro"},{"id":"16092","name":".kim 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.kim"},{"id":"16055","name":".work 域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.work"},{"id":"16011","name":".red 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.red"},{"id":"17031","name":".group 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.group"},{"id":"16062","name":".store 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.store"},{"id":"15427","name":".pub 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.pub"},{"id":"15107","name":".live 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.live"},{"id":"15097","name":".wiki 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.wiki"},{"id":"15098","name":".design 英文域名","url":"http://wanwang.aliyun.com/domain/newgtld/?#.design"},{"id":"14581","name":".gov.cn 政府域名","url":"http://wanwang.aliyun.com/domain/govcn/"},{"id":"12201","name":".biz 商务域名","url":"http://wanwang.aliyun.com/domain/biz/"},{"id":"12021","name":".org 英文域名","url":"http://wanwang.aliyun.com/domain/org/"},{"id":"10961","name":".name 名字域名","url":"http://wanwang.aliyun.com/domain/name/"},{"id":"has2acc","name":".cc 公司域名","url":"http://wanwang.aliyun.com/domain/cc/"},{"id":"has2atv","name":".tv 媒体域名","url":"http://wanwang.aliyun.com/domain/tv/"},{"id":"12081","name":".mobi 手机域名","url":"http://wanwang.aliyun.com/domain/mobi/"},{"id":"17101","name":".在线","url":"http://wanwang.aliyun.com/domain/newgtld/?#.在线"},{"id":"17041","name":".网址","url":"http://wanwang.aliyun.com/domain/newgtld/?spm=5176.8061372.1001.42.7a1af8ff8vAnNh#.%E7%BD%91%E5%9D%80"},{"id":"17121","name":".网店","url":"http://wanwang.aliyun.com/domain/newgtld/?spm=5176.8061372.1001.48.5ba9fb5W6eBzI#.%E7%BD%91%E5%BA%97"},{"id":"17102","name":".中文网","url":"http://wanwang.aliyun.com/domain/newgtld/?#.中文网"},{"id":"12903","name":".中国","url":"http://wanwang.aliyun.com/domain/zhchina/"},{"id":"20149","name":".公司","url":"http://wanwang.aliyun.com/domain/company/"},{"id":"20149","name":".网络","url":"http://wanwang.aliyun.com/domain/network/"},{"id":"12256","name":".我爱你","url":"http://wanwang.aliyun.com/domain/newgtld/?spm=5176.8061372.1001.41.1217dcecMQFsgE#.%E6%88%91%E7%88%B1%E4%BD%A0"},{"id":"12260","name":".集团","url":"http://wanwang.aliyun.com/domain/newgtld/?spm=5176.8061372.1001.42.1d0fc1e72smKeE#.%E9%9B%86%E5%9B%A2"},{"id":"4agb","name":"中文.com","url":"http://wanwang.aliyun.com/domain/zhcom/"},{"id":"12904","name":"中文.cn","url":"http://wanwang.aliyun.com/domain/zhcn/"},{"id":"4agb","name":"中文.net","url":"http://wanwang.aliyun.com/domain/zhnet/"},{"id":"12461","name":"中文.cc","url":"http://wanwang.aliyun.com/domain/zhcc/"},{"id":"12462","name":"中文.tv","url":"http://wanwang.aliyun.com/domain/zhtv/"},{"id":"12561","name":"中文.biz","url":"http://wanwang.aliyun.com/domain/zhbiz/"},{"id":"4agb-cn","name":"中文.中国","url":"http://wanwang.aliyun.com/domain/zhchina/"},{"id":"20148","name":"中文.公司","url":"http://wanwang.aliyun.com/domain/zhcompany/"},{"id":"20148","name":"中文.网络","url":"http://wanwang.aliyun.com/domain/zhnetwork/"},{"id":"16013","name":".bid 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16012","name":".loan 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15071","name":".click 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15074","name":".gift 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15075","name":".pics 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15076","name":".photo 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16095","name":".men 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"17051","name":".pw 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15081","name":".news 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15082","name":".video 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15083","name":".win 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15084","name":".party 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15085","name":".date 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15086","name":".trade 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15087","name":".science 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15093","name":".website 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15094","name":".space 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15095","name":".press 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15422","name":".rocks 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15423","name":".band 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15424","name":".engineer 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15425","name":".market 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15428","name":".social 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15429","name":".software 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15430","name":".lawyer 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15108","name":".studio 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16051","name":".mom 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16052","name":".lol 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16053","name":".game 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16093","name":".games 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"15073","name":".help 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"13701","name":".me 我的域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"17922","name":".co 公司域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"16122","name":".vc 国别域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"17923","name":".so国别域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"14461","name":".tel 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"12741","name":".hk 英文域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"13321","name":".asia 亚洲域名","url":"http://wanwang.aliyun.com/domain/"},{"id":"12742","name":"中文.hk","url":"http://wanwang.aliyun.com/domain/"}]

In [3]:
domain_def_list[1]
domain_def_list[1]['id']
len(domain_def_list)

{'id': '15201',
 'name': '.net 英文域名',
 'url': 'http://wanwang.aliyun.com/domain/net/'}

'15201'

94

In [4]:
def parse_jsonp(jsonp_str): ##jsonp处理
    try:
        return json.loads( re.search('^[^(]*?\((.*)\)[^)]*$', jsonp_str).group(1) )
    except:
        raise ValueError('Invalid JSONP')


In [5]:
def load_price(id):
    domain_load='''[{"productID":"%s","period":12,"amount":1,"action":"activate"},
    {"productID":"%s","period":24,"amount":1,"action":"activate"}]
    ''' %(id,id)


    payload = {'callback': 'jQuery111109917434921775314_1536892807565', 'data': domain_load,'_': time.time() }
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36',
               'referer':'https://wanwang.aliyun.com/help/price.html',
               'dnt':'1',
               'cookie':'aliyun_choice=CN; cna=whgiFOGePFgCAQ4RFiWJ++g4; ww_csrf=mNeuIX%2BW%2BM9YJWRFSylXTi482qFOnsTYCboXy8vdMUd0BerjRvWlXkODZkoD9fcxEWCTQdDx%2Bv4rJVI1420OLYy%2BiEmv9cpruR0tjgueQ6gRf3pzWM6FpB3n3svirgY54NAyF%2FmDN1Qwm3yFN5Z0hw%3D%3D; stk_lgn=98af63a6-8ab8-4f02-903a-5b2d65a06b60; JSESSIONID=D049DBF777773E9C51F2D8C418102B50; isg=BIuLzP6rVMkjp4idx3rxQ6YdGi9_Ksnoz6jQRf2IZ0ohHKt-hfAv8il68lxXPPea'
               ,'authority':'netcn.console.aliyun.com'}
    r = requests.get('https://netcn.console.aliyun.com/core/product/infostatic',params=payload,headers=headers)

    # print(r.url)  ##反爬虫比较厉害，抓不到数据
#     print(r.status_code)
    # r.headers
#     print( parse_jsonp(r.text) )
    return parse_jsonp(r.text)

In [8]:
def prase_price_json(js):
    try:
        dsc=js['model'][0]['productName']
        money_1=js['model'][0]['money']
        money_2=js['model'][1]['money'] -js['model'][0]['money']
        action=js['model'][0]['action']
        return dsc,money_1,money_2,action
    except:
        raise IndexError(js)

In [9]:
p =load_price('15201')
print(prase_price_json(p))

('.net 英文域名', 69.0, 69.0, 'activate')


In [12]:
list_p = []

for d in domain_def_list:
    p =load_price(d['id'])
    if len(p['model'])>0:
        dsc,money_1,money_2,action =prase_price_json(p)
        list_p.append({'dsc':dsc,'money_1':money_1,'money_2':money_2,'action':action})

In [34]:
df=pd.DataFrame(list_p)
df=df.drop_duplicates(['dsc'])

In [39]:
df.shape
df.dsc.unique()

(55, 4)

array(['.com 英文域名', '.net 英文域名', '.cn 英文域名', '.info 英文域名', '.xin 域名',
       '.club 域名', '.xyz 域名', '.ltd 域名', '.wang 域名', '.top 域名', '.vip 域名',
       '.beer域名', '.ink 域名', '.site 域名', '.shop 域名', '.fun 域名', '.link 域名',
       '.online 域名', '.tech 域名', '.ren 域名', '.pro 域名', '.kim 域名',
       '.work 域名', '.red 域名', '.group域名', '.store 域名', '.pub 域名',
       '.live 域名', '.wiki 域名', '.design 域名', '.gov.cn英文域名', '.biz 英文域名',
       '.org 英文域名', '.name 英文域名', '.cc 英文域名', '.tv 英文域名', '.mobi域名',
       '.在线 域名', '.网址域名', '.网店 域名', '.中文网 域名', '.中国 英文域名', '.公司/网络 英文域名',
       '.我爱你 域名', '.集团 域名', '.com/net 中文域名', '.cn 中文域名', '.cc 中文域名',
       'tv 中文域名', 'biz中文域名', '国内中文域名', '.公司/网络 中文域名', '.co国别域名', '.hk英文域名',
       '.hk中文域名'], dtype=object)

In [40]:
df
df.to_csv('ali-domain-price.csv',header=True)

,action,dsc,money_1,money_2
0,activate,.com 英文域名,55.0,69.0
1,activate,.net 英文域名,69.0,69.0
2,activate,.cn 英文域名,29.0,39.0
3,activate,.info 英文域名,9.0,85.0
4,activate,.xin 域名,45.0,39.0
5,activate,.club 域名,14.0,59.0
6,activate,.xyz 域名,4.0,45.0
7,activate,.ltd 域名,16.0,35.0
8,activate,.wang 域名,19.0,32.0
9,activate,.top 域名,5.0,26.0
